In [ ]:
%matplotlib inline
import sys, os
import numpy as np
import inspect
from openxrd import find_peaks_2d
from openxrd import find_peaks_1d
from openxrd import fit_single
from openxrd import get_fit_all_1d
from openxrd import get_fit_all_2d
from openxrd import fits_to_csv_multitype
from openxrd import fit_data_to_csv
from openxrd import BrukerData
from openxrd import fit_multipeak
from openxrd import fits_to_csv
from openxrd import csv_append_col

from datasetmetta import get_name_data

import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.patheffects as path_effects
from mpl_toolkits.axes_grid1 import make_axes_locatable

import csv
from lmfit import models
from lmfit import lineshapes

import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib as mpl


# STYLES ########################
axlabels = {'family':               'sans',
            'color':                'black',
            'weight':               'normal',
            'size':                 12,
            }
titles = {'family':                 'sans',
          'color':                  'black',
          'weight':                 'normal',
          'size':                   14,
          }
labels = {'family':                 'sans',
          'fontname':               'DejaVu Sans',
          'color':                  '#000000',
          'weight':                 'normal',
          'size':                   9,
          'verticalalignment':      'center',
          'horizontalalignment':    'right'
          }
# ################################

def get_scalarmap(heat, hrng=None, buf=None):
    if not buf:
        buf = (max(heat)-min(heat))*0.2
    if not hrng:
        hrng = [min(heat), max(heat)]
    cmap = plt.cm.gnuplot
    cNorm  = colors.Normalize(vmin=hrng[0] - buf, vmax=hrng[1] + buf)
    return cmx.ScalarMappable(norm=cNorm,cmap=cmap)
                    
def plot_vals(axis, x, y, heat, form, rot, valign='bottom', hrng=None, buf=None, fontdict=labels):
    scalarMap = get_scalarmap(heat, hrng, buf)
    txt = []
    for i, val in enumerate(heat):
        colorVal = scalarMap.to_rgba(val)
        
        # add some spacing b/t text
        x_r = -np.sin(rot[i])*1.5
        y_r = np.cos(rot[i])*1.5
        if valign == 'bottom':
            x_r += x[i]
            y_r += y[i]
        elif valign == 'top':
            x_r = x[i] - x_r
            y_r = y[i] - y_r
        else:
            x_r = x[i]
            y_r = y[i]            
        txt.append(axis.text(x_r, y_r,
                            form.format(val),
                            fontdict=fontdict, 
                            rotation=np.degrees(rot[i]), 
                            horizontalalignment='center',
                            verticalalignment=valign,
                            color=colorVal,
                            rotation_mode="anchor"))
                            #bbox={'pad':30, 'fc':'none', 'ec':'none'}))
            
    #return txt
      
def plot_arrows(axis, x, y, x_d, y_d, heat, hrng=None, buf=None):
    scalarMap = get_scalarmap(heat, hrng, buf)
    arrows = []
    for i, val in enumerate(heat):
        colorVal = scalarMap.to_rgba(val)
        arrows.append(axis.arrow(x[i], y[i],
                                 x_d[i], y_d[i], 
                                 head_width=2, head_length=5,
                                 color=colorVal))
        #cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap,
        #                        norm=cNorm,orientation='vertical',
        #                        spacing='proportional') 
    return arrows


def plot_wafer(mplax, flip=0):
    """Plot 100mm wafer
        mplax - matplotlib subplot"""
    mplax.plot([51,51],[-51,51], color="w")
    mplax.plot([-51,51],[51,51], color="w")
    
    if flip:
        mplax.plot([-15,15],[48,48], color="black")
        wafer = Arc((0,0),height=100,width=100,angle=0,
                theta1=107,theta2=73,color="black", lw=1.5)
    else:
        mplax.plot([-15,15],[-48,-48], color="black")
        wafer = Arc((0,0),height=100,width=100,angle=0,
                theta1=287,theta2=253,color="black", lw=1.5)

    mplax.add_patch(wafer)
    mplax.axis('off')
    return mplax

def get_rc_x(rng):
    """ return np.array of Rocking curve x values
        rng - BrukerData.rng"""
    ##need to make do either rc or 2th and add to BrukerData class
    th_0 = rng.metta['start_theta']
    twoth_0 = rng.metta['start_2th']
        
    step_size = rng.metta['step_size']
    steps = rng.metta['steps']
        
    #this will center each scan based on input 2th
    length = step_size*steps
    end = th_0 + length - twoth_0/2
    start = th_0 -twoth_0/2

    return np.linspace(start, end, steps)   


def gettilt_plotrc(data, mplax_rc=None, flip=0):
    c_x = np.empty((0,))
    c_y = np.empty((0,))
    fwhm1 = np.empty((0,))
    fwhm2 = np.empty((0,))
    d_x = np.empty((0,))
    d_y = np.empty((0,))
    rots = np.empty((0,))
    js = np.empty((0,))

    for j, rng in enumerate(data.rngs):

        y = np.asarray(rng.counts_data)
        x = get_rc_x(rng)
    
        #fit PseudoVoigt
        out = fit_single(x, y)

        #plot rc
        if mplax_rc:
            mplax_rc.plot(x, y, 'C'+str(j), linewidth=1, label=str(j+1)) #plot
            mplax_rc.plot(x, out.best_fit,  'C'+ str(j), linewidth=1,)
    
        #get vals

        # measurement positions (x and y are reversed and y is neg)
        rot = rng.metta['drive_phi'] 
        
        # Handel fliped wafer position
        if flip:
            f = -1
        else:
            f = 1
        # Arrow vector
        if rot == 90:
            c_y = np.append(c_y, f*-out.report['center'])
            d_x = np.append(d_x, f*rng.metta['drive_y'])
            d_y = np.append(d_y, f*-rng.metta['drive_x'])
            fwhm1 = np.append(fwhm1, out.report['fwhm'])
        else:    
            c_x = np.append(c_x, f*-out.report['center'])
            fwhm2 = np.append(fwhm2, out.report['fwhm'])
        rots = np.append(rots, rot-90)
        js = np.append(js, j)
    c_c = np.sqrt((c_x)**2 + (c_y)**2)


    
    return mplax_rc, {'c_y': c_y, 'c_x': c_x, 'd_x': d_x, 'd_y': d_y, 
                      'fwhm1': fwhm1, 'fwhm2': fwhm2, 'c_c': c_c, 'rots': rots, 'js': js}


def plot_tilts(mplax_wf, dat, c_hrng, f_hrng, fwhm=0):
    #plot tilts
    arrows = plot_arrows(mplax_wf, dat['d_x'], dat['d_y'], 
                         dat['c_x']*5*dat['c_c']**2,
                         dat['c_y']*5*dat['c_c']**2,
                         dat['c_c'], c_hrng)       
    if fwhm:
        f_dat = dat['fwhm2']
    else:
        f_dat = dat['fwhm1']
        
    txt1 = plot_vals(mplax_wf, dat['d_x'], dat['d_y'], f_dat,
                     'fwhm:\n{0:.3f}\u00b0', np.arctan(dat['c_y']/dat['c_x']), hrng=f_hrng)
            
    #deg tilt calc
    txt2 = plot_vals(mplax_wf, dat['d_x'], dat['d_y'], dat['c_c'],
                     'tilt:\n{0:.3f}\u00b0', np.arctan(dat['c_y']/dat['c_x']), 'top', c_hrng) 
    return mplax_wf

# MAIN
def multi(files, order, c_hrng=None, f_hrng=None, fwhm=0):
    #PLOT Setup
    gs_rc = plt.GridSpec(2, 2, top=0.95, bottom=0.54, right=0.75, wspace=0.3, hspace=0.4)
    gs_wf = plt.GridSpec(2, 2, top=0.46, bottom=0.05, right=0.75, wspace=0, hspace=0)
    gs_lgd = plt.GridSpec(1, 1, top=0.95, bottom=0.05, right=1, wspace=0, hspace=0)
    fig = plt.figure(figsize=(8, 12), dpi=100)
    ax_lgd = fig.add_subplot(gs_lgd[0])
    ax_lgd.axis('off')


    # set up axis
    ax_rc = [None]*4
    ax_wf = [None]*4 
    for i in range(2):
        for j in range(2):
            ax_wf[i*2+j] = fig.add_subplot(gs_wf[i,j])
            ax_wf[i*2+j] = plot_wafer(ax_wf[i*2+j], 1-i)
            ax_wf[i*2+j].set_title('P%d'% (order[i*2+j]), y=0.95)
            
            ax_rc[i*2+j] = fig.add_subplot(gs_rc[i,j])
            ax_rc[i*2+j].set_title('P%d'% (order[i*2+j]), y=1)
            # axis Lables
            ax_rc[i*2+j].set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
            ax_rc[i*2+j].set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
            ax_rc[i*2+j].set_yticklabels([])
            # Set axis tick labels font
            for label in (ax_rc[i*2+j].get_xticklabels() + ax_rc[i*2+j].get_yticklabels()):
                for prop in axlabels:
                    getattr(label, 'set_' + prop)(axlabels[prop])

    if fwhm:
        fh = 'fwhm2'
    else:
        fh = 'fwhm1'                 
                    
    # Get Data      
    tilt_dat = [{'c_c': [-1], fh: [-1]}]*4
    for i, p in enumerate(order):
        if not files[p-1]:
            continue
        data = BrukerData(files[p-1])
        ax_rc[i], tilt_dat[i] = gettilt_plotrc(data, ax_rc[i], flip=1-i//2)
    
    if not c_hrng:   
        tilts = [x for d in tilt_dat for x in d['c_c'] if x > 1]
        c_hrng = [min(tilts), max(tilts)]
    if not f_hrng:
        fwhms = [x for d in tilt_dat for x in d[fh] if x > 1]
        f_hrng = [min(fwhms), max(fwhms)]

    print('tilt range[\u00b0]: [{0:.3f}, {1:.3f}]'.format(c_hrng[0], c_hrng[1]))
    print('fwhm range[\u00b0]: [{0:.3f}, {1:.3f}]'.format(f_hrng[0], f_hrng[1]))

    for i, dat in enumerate(tilt_dat):
        if dat['c_c'][0] < 0:
            continue
        ax_wf[i] = plot_tilts(ax_wf[i], dat, c_hrng, f_hrng, fwhm)    

    handles, labels = ax_rc[0].get_legend_handles_labels()
    lgd = fig.legend(handles, labels, loc=7,bbox_to_anchor=(0.80, 0.75))  
    plt.show()
    
def no_map(file, name):
    #PLOT Setup
    fig = plt.figure(figsize=(6, 6), dpi=100)
    ax_rc = fig.add_subplot(111)
    ax_rc.set_title(name)
    # axis Lables
    ax_rc.set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
    ax_rc.set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
    ax_rc.set_yticklabels([])
    # Set axis tick labels font
    for label in (ax_rc.get_xticklabels() + ax_rc.get_yticklabels()):
        for prop in axlabels:
            getattr(label, 'set_' + prop)(axlabels[prop])

    # Get Data
    data = BrukerData(file)
    for j, rng in enumerate(data.rngs):
        y = np.asarray(rng.counts_data)
        x = get_rc_x(rng)
        #fit PseudoVoigt
        out = fit_single(x, y)
        #plot rc
        ax_rc.plot(x, y, 'C'+str(j), linewidth=1) #plot
        ax_rc.plot(x, out.best_fit,  'C'+ str(j), linewidth=1,)
        #d_x = np.append(d_x, f*rng.metta['drive_y'])
        #d_y = np.append(d_y, f*-rng.metta['drive_x'])
        fwhm = out.report['fwhm']
        center = out.report['center']
        # plot fwhm line
        x1 = np.abs(x-center-fwhm/2).argmin()
        x2 = np.abs(x-center+fwhm/2).argmin()
        ax_rc.plot([center-fwhm/2, center+fwhm/2],
                   [out.best_fit[x1], out.best_fit[x2]],
                   'C'+str(j), linewidth=1, linestyle=':')
        txt = ax_rc.text(center, out.best_fit[x1],
                            'fwhm:\n{0:.3f}\u00b0'.format(fwhm),
                            fontdict=labels, 
                            rotation=np.degrees(0), 
                            horizontalalignment='center',
                            verticalalignment='bottom',
                            rotation_mode="anchor")
        txt2 = ax_rc.text(center, out.best_fit[x1],
                            'center:\n{0:.3f}\u00b0'.format(center),
                            fontdict=labels, 
                            rotation=np.degrees(0), 
                            horizontalalignment='center',
                            verticalalignment='top',
                            rotation_mode="anchor")
    #ax_rc.legend(loc=1)#,bbox_to_anchor=(1, 0.75))  
    plt.show()        
    
def single(file, name, c_hrng=None, f_hrng=None, fwhm=0):
    #PLOT Setup
    fig = plt.figure(figsize=(6, 12), dpi=100)
    ax_rc = fig.add_subplot(211)
    ax_wf = fig.add_subplot(212)

    ax_wf = plot_wafer(ax_wf, 0)
    ax_wf.set_title('')
    
    ax_rc.set_title(name)
    # axis Lables
    ax_rc.set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
    ax_rc.set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
    ax_rc.set_yticklabels([])
    # Set axis tick labels font
    for label in (ax_rc.get_xticklabels() + ax_rc.get_yticklabels()):
        for prop in axlabels:
            getattr(label, 'set_' + prop)(axlabels[prop])
            
            
    if fwhm:
        fh = 'fwhm2'
    else:
        fh = 'fwhm1' 
        
    # Get Data
    data = BrukerData(file)
    ax_rc, dat = gettilt_plotrc(data, ax_rc, flip=0)
    
    if not c_hrng:
        c_hrng = [min(dat['c_c']), max(dat['c_c'])]
    if not f_hrng:
        f_hrng = [min(dat[fh]), max(dat[fh])]
    
    print('tilt range[\u00b0]: [{0:.3f}, {1:.3f}]'.format(c_hrng[0], c_hrng[1]))
    print('fwhm range[\u00b0]: [{0:.3f}, {1:.3f}]'.format(f_hrng[0], f_hrng[1]))
    
    ax_wf = plot_tilts(ax_wf, dat, c_hrng, f_hrng, fwhm)    
    ax_rc.legend(loc=1)#,bbox_to_anchor=(1, 0.75))  
    plt.show() 
    return dat
            
def positions(file):
    #PLOT Setup
    fig = plt.figure(figsize=(6, 6), dpi=100)
    ax_rc = None
    ax_wf = fig.add_subplot(111)
    ax_wf = plot_wafer(ax_wf, 0)
    ax_wf.set_title('')
    # Get Data
    data = BrukerData(file)
    ax_rc, dat = gettilt_plotrc(data)
    d_x = [x for x in dat['d_x'] for z in [x, x]]
    d_y = [x for x in dat['d_y'] for z in [x, x]]
    rot = [np.radians(x) for x in dat['rots']]
    txt3 = plot_vals(ax_wf, d_x , d_y, dat['js'],
                     'pos:{0:.0f}', rot, valign='center', fontdict=titles)  
    plt.show()

# Get data
root = '/mnt/W/Austin_Fox/'
rootf = root + 'XRD/data/'

In [ ]:
from lmfit.models import PseudoVoigtModel
# File used as reference    
file = rootf + '8719_map_RC.raw' 
data = BrukerData(file)
ax = None
ax, dat2 = gettilt_plotrc(data, ax, flip=0)
for key in dat2:
    print(key, dat2[key])

    #'c_y': c_y, 'c_x': c_x, 'd_x': d_x, 'd_y': d_y, 
    #                   'fwhm1': fwhm1, 'fwhm2': fwhm2, 'c_c': c_c, 'rots': rots, 'js': js}

file = rootf + '5582_map_RC.raw'
ax = None
# Get Data
data = BrukerData(file)
for j, rng in enumerate(data.rngs):

        y = np.asarray(rng.counts_data)
        x = get_rc_x(rng)
    
        #fit PseudoVoigt        
        out = fit_single(x, y, plot=True)
        print(out.report['fwhm'])
        out = fit_multipeak(x, y, 'Test', models=[PseudoVoigtModel, PseudoVoigtModel,
                                                  ], plot=True)
        print(out.report['mod_0_fwhm'], out.report['mod_1_fwhm'], out.report['mod_0_center'], out.report['mod_1_center'])
        """
        #plot rc
        if mplax_rc:
            mplax_rc.plot(x, y, 'C'+str(j), linewidth=1, label=str(j+1)) #plot
            mplax_rc.plot(x, out.best_fit,  'C'+ str(j), linewidth=1,)
    
        #get vals

        # measurement positions (x and y are reversed and y is neg)
        rot = rng.metta['drive_phi'] 
        
        # Handel fliped wafer position
        if flip:
            f = -1
        else:
            f = 1
        # Arrow vector
        if rot == 90:
            c_y = np.append(c_y, f*-out.report['center'])
            d_x = np.append(d_x, f*rng.metta['drive_y'])
            d_y = np.append(d_y, f*-rng.metta['drive_x'])
            fwhm1 = np.append(fwhm1, out.report['fwhm'])
        else:    
            c_x = np.append(c_x, f*-out.report['center'])
            fwhm2 = np.append(fwhm2, out.report['fwhm'])
        rots = np.append(rots, rot-90)
        js = np.append(js, j)
    c_c = np.sqrt((c_x)**2 + (c_y)**2)


    
    return mplax_rc, {'c_y': c_y, 'c_x': c_x, 'd_x': d_x, 'd_y': d_y, 
                      'fwhm1': fwhm1, 'fwhm2': fwhm2, 'c_c': c_c, 'rots': rots, 'js': js}
    """